<a href="https://colab.research.google.com/github/ichekhovskikh/doc2vec/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install pymorphy2[fast]

import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 70 текстов.

И мы проверим нашу модель на глаз, используя тестовый корпус, который содержит 7 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать)

In [0]:
#@title Введите путь к файлам исходной базы статей:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/train_corpus.json' #@param {type: "string"}
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/test_corpus.json' #@param {type: "string"}

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

Лемматизация каждого слова статьи:

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = list(lemmatize(normalized_text))
    normalized_text = remove_stopwords(normalized_text)
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path, tokens_only=False):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            normalized_text = advanced_preprocess(article['text'])
            if tokens_only:
                yield normalized_text
            else:
                yield gensim.models.doc2vec.TaggedDocument(normalized_text, article['tags'])

Получение исходного текста статьи по индексу:

In [0]:
def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

Получение исходного текста статьи по идентификатору:

In [0]:
def get_article_text_by_id(id, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for index in range(len(corpus)):
          if (corpus[index]['id'] == id):
            return corpus[index]['text']
        return ""

Получение категории статьи по индексу:

In [0]:
def get_article_class_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['class_name']

Получение категории статьи по идентификатору:

In [0]:
def get_article_class_by_id(id, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for index in range(len(corpus)):
          if (corpus[index]['id'] == id):
            return corpus[index]['class_name']
        return ""

In [0]:
train_corpus = list(read_corpus(train_path))
test_corpus = list(read_corpus(test_path, tokens_only=True))

Давайте посмотрим на учебный корпус:

In [10]:
print(train_corpus[:2])

[TaggedDocument(words=['качество', 'теплоноситель', 'применяться', 'жидкость', 'который', 'позволять', 'быстро', 'охлаждать', 'воздух', 'принцип', 'работа', 'система', 'охлаждение', 'достаточно', 'простой', 'понятный', 'тепло', 'процессор', 'передаваться', 'тепловой', 'обменник', 'вода', 'нагреваться', 'перемещаться', 'радиатор', 'испариться', 'список', 'литература', 'references', 'электронный', 'ресурс', 'режим', 'доступ', 'vyboroved', 'ru', 'https', 'vyboroved', 'ru', 'reyting', 'luchshie', 'sistemy', 'okhlazhdeniya', 'kompyutera', 'html', 'дата', 'обращение', 'электронный', 'ресурс', 'режим', 'доступ', 'ekatalog', 'http', 'ek', 'ua', 'htm', 'дата', 'обращение', 'электронный', 'ресурс', 'мастер', 'компьютерный', 'помощь', 'режим', 'доступ', 'masterservis', 'ru', 'http', 'masterservis', 'ru', 'vodyanoe', 'ohlazhdenie', 'dlya', 'kompyutera', 'html', 'дата', 'обращение', 'электронный', 'ресурс', 'fb', 'ru', 'режим', 'доступ', 'http', 'fb', 'ru', 'article', 'kakoe', 'ohlajdenie', 'dlya',

Корпус тестирования выглядит так:

In [11]:
print(test_corpus[:2])

[['technical', 'science', 'удк', 'мальков', 'тверская', 'государственный', 'технический', 'университет', 'doi', 'метод', 'построение', 'классификатор', 'технический', 'документация', 'malkov', 'tver', 'state', 'technical', 'university', 'methods', 'for', 'constructing', 'classifier', 'technical', 'documentations', 'аннотация', 'приводиться', 'постановка', 'задача', 'классификация', 'применительно', 'построение', 'система', 'автоматический', 'классификация', 'текстовый', 'документ', 'рассматриваться', 'метод', 'решение', 'задача', 'автоматический', 'классификация', 'текстовый', 'документ', 'приводиться', 'подход', 'построение', 'система', 'автоматический', 'классификация', 'abstract', 'the', 'formulation', 'of', 'the', 'classification', 'problem', 'for', 'the', 'construction', 'of', 'the', 'system', 'of', 'automatic', 'classification', 'of', 'text', 'documents', 'is', 'given', 'methods', 'of', 'solving', 'the', 'problem', 'of', 'automatic', 'classification', 'of', 'text', 'documents', '

Обратите внимание, что корпус тестирования представляет собой просто список списков и не содержит никаких тегов.

# Обучение модели
## Создание объекта Doc2Vec
Теперь мы создадим модель Doc2Vec с векторным размером 80 слов и перебираем учебный корпус 100 раз. Мы устанавливаем минимальную длину слова равной двум, чтобы отбрасывать слова с очень малым количеством вхождений.

Однако это очень маленький набор данных (70 документов) с короткими текстами (несколько сотен слов). 

Словарь содержит в себе все уникальные слова, извлеченных из учебного корпуса.

In [0]:
#@title Укажите параметры обучения модели:
vector_size = 80 #@param
window =  2 #@param
epochs =  100 #@param
alpha = 0.001 #@param
learning_method = "PV-DM" #@param ["PV-DM", "PV-DBOW"] {type:"raw"}

dm = 1
if (learning_method == "PV-DBOW"):
    dm = 0

In [64]:
model = gensim.models.doc2vec.Doc2Vec(
    vector_size=vector_size, 
    dm=dm,
    window=window, 
    min_count=min_count, 
    alpha=alpha, 
    epochs=epochs)

model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 6min 32s, sys: 1.75 s, total: 6min 34s
Wall time: 3min 25s


## Inferring a Vector
Важно отметить, что теперь вы можете вывести вектор для любого фрагмента текста без необходимости переобучать модель, передав список слов в функцию model.infer_vector. Затем этот вектор можно сравнить с другими векторами по косинусной близости.

In [0]:
text = 'Горностай небольшой зверек семейства куньих очень ценится'
model.infer_vector(advanced_preprocess(text))

array([-0.04096178, -0.18379952, -0.11860437, -0.20194705, -0.50240123,
        0.1839197 , -0.15432355,  0.72807366,  0.5409689 , -0.4779142 ,
       -0.10427655,  0.20272826,  0.00589776,  0.05693987,  0.16688912,
       -0.42739728, -0.01697599, -0.07323613,  0.26289058,  0.64651096,
        0.17578113, -0.28664327, -0.07712755,  0.03304586, -0.3750441 ,
       -0.1947108 , -0.13230076,  0.13952   , -0.02209855, -0.19145404,
        0.31799352, -0.17331338, -0.15030901, -0.04337816,  0.37577283,
       -0.14438367, -0.15898366,  0.16706972,  0.08218319, -0.24977092,
       -0.43249604, -0.38224697,  0.16896726, -0.02178737,  0.284938  ,
       -0.21531668,  0.13852073, -0.21743523, -0.2834976 ,  0.04890248,
       -0.03676964,  0.3413355 ,  0.1220827 ,  0.3642785 ,  0.07248931,
       -0.32313284, -0.81035566,  0.074465  ,  0.23671976,  0.09117885,
       -0.78747183,  0.03215976, -0.19274847,  0.43228692, -0.01104711,
        0.29571605,  0.00713215, -0.5716812 ,  0.19893852,  0.33

Обратите внимание, что, поскольку лежащие в основе алгоритмы обучения представляют собой подходы на основе итеративной аппроксимации, в которой используется внутренняя рандомизация, таким обрахрм, повторные выводы одного и того же текста будут возвращать слегка разные векторы.

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [49]:
doc_id = random.randint(0, len(train_corpus) - 1)

inferred_vector = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([inferred_vector], topn=5)

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}) «{}»: «{}»\n'.format(doc_id, get_article_class_by_id(doc_id, train_path), get_article_text_by_index(doc_id, train_path)))
for label, index in [('1)', 0), ('2)', 1), ('3)', 2), ('4)', 3), ('5)', 4)]:
    print(u'%s %s «%s»: «%s»\n' % (label, sims[index], get_article_class_by_id(int(sims[index][0]), train_path), get_article_text_by_id(int(sims[index][0]), train_path)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ТЕКСТ ИСХДНОГО ДОКУМЕНТА (56) «машинное обучение»: « ﻿Применение методов машинного обучения при работе с литературными источниками Артюхин Валерий Викторович к. э. н., доц., ведущий научный сотрудник Центра анализа и управления рисками, Федеральное государственное бюджетное учреждение «Всероссийский научно-исследовательский институт по проблемам гражданской обороны и чрезвычайных ситуаций» МЧС России (федеральный центр науки и высоких технологий), 121352, г. Москва, ул. Давыдковская, д.7, ФГБУ ВНИИ ГОЧС (ФЦ), 6 НИЦ, &#43;7 (499) 2332570, ikshot@mail.ru Аннотация В статье рассматриваются два примера применения методов машинного обучения, позволяющих существенно облегчить работу исследователя или преподавателя при поиске и анализе литературных источников. Кластеризация книг позволяет получить представление о структуре и содержании публикационной активности по определенной тематике. С другой стороны, применение описанной вариации алгоритма Луна для ранжирования предложений позволяет выдел

Обратите внимание, что наиболее похожий документ (как правило, тот же текст) имеет параметр сходства, приближающийся к единице. 

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [60]:
from sklearn.metrics import f1_score

test_size = len(test_corpus)

predicted_classes = [] 
test_classes = []

for doc_index in range(test_size):
    inferred_vector = model.infer_vector(test_corpus[doc_index])
    sims = model.docvecs.most_similar([inferred_vector], topn=5)

    test_article_class = get_article_class_by_index(doc_index, test_path)
    print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}) «{}»: «{}»\n'.format(doc_id, test_article_class, get_article_text_by_index(doc_index, test_path)))
    for label, index in [('1)', 0), ('2', 1), ('3)', 2), ('4)', 3), ('5)', 4)]:
        predicted_article_class = get_article_class_by_id(int(sims[index][0]), train_path)
        print(u'%s %s «%s»: «%s»\n' % (label, sims[index], predicted_article_class, get_article_text_by_id(int(sims[index][0]), train_path)))
        predicted_classes.append(predicted_article_class)
        test_classes.append(test_article_class)

print('f1score: {}'.format(f1_score(test_classes, predicted_classes, average='macro')))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


f1score: 0.8809235470994139


# Поиск похожих научных документов
Выполните поиск похожих научных статей

In [0]:
#@title Укажите путь к тексту статьи в формате *.txt или введите текст статьи:
article_text = '' #@param {type: "string"}
article_path = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/example1.txt' #@param {type: "string"}

if (article_text == ''):
    with urllib.request.urlopen(article_path) as article_url:
      article_text = article_url.read().decode()


In [0]:
normalized_text = advanced_preprocess(article_text)
inferred_vector = model.infer_vector(normalized_text)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА «{}»\n'.format(article_text)
for label, index in [('1)', 0), ('2', 1), ('3)', 2), ('4)', 3)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], get_article_text_by_id(int(sims[index][0]), train_path)))